# MYMail_review – Notas / comandos rápidos

Este notebook sirve como recordatorio de comandos para:
- Subir nuevos ficheros a **entrada**
- Gestión de **usuarios** (incluyendo actualización de contraseña)


## 1) Subir un nuevo Excel a ENTRADA (blob + tabla)

Carga el Excel en el contenedor `entrada` y crea registros en la tabla `entrada`.


In [ ]:
!python scripts/load_entrada.py --excel "C:\\ruta\\Validados_V3.xlsx" --sheet "1 dic - 8 dic"

In [ ]:
!python scripts/load_entrada.py --excel "Validados_V3.xlsx" --sheet "1 dic - 8 dic" --partition "default"

In [ ]:
!python scripts/load_entrada.py --excel "Validados_V3.xlsx" --sheet "1 dic - 8 dic" --replace

Flags útiles:
- `--replace` borra la partición antes de cargar
- `--partition` PartitionKey destino (por defecto: `default`)


## 2) Gestión de usuarios (Azure Table Storage)

### 2.1 Crear/actualizar usuario


In [ ]:
!python scripts/manage_users.py add --username "usuario1" --password "MiPassword" --role revisor

In [ ]:
!python scripts/manage_users.py add --username "admin" --password "MiPassword" --role admin

### 2.2 Cambiar rol


In [ ]:
!python scripts/manage_users.py set-role --username "usuario1" --role admin

### 2.3 Actualizar contraseña


In [ ]:
!python scripts/manage_users.py set-password --username "usuario1" --password "NuevaPassword"

### 2.4 Listar usuarios


In [2]:
!python scripts/manage_users.py list

admin	Administrador	active=1
mangel	Administrador	active=1
miguel.galan	Revisor	active=1


## Requisitos

- Variables en `.env` / `config.py` (mínimo `AZURE_STORAGE_CONNECTION_STRING`).
- Dependencias instaladas: `pip install -r requirements.txt`


## 3) Test de Azure OpenAI (legacy)

Usa el script `scripts/test_azure_openai.py` para validar que el endpoint + deployment + api-version funcionan.


In [3]:
! .\\.venv\\Scripts\\python.exe scripts/test_azure_openai.py --prompt "Responde exactamente: OK" --temperature 1 --max-tokens 256

OK
